In [ ]:
#   Hacer una copia local del las carpetas

def copy_cloned_folder(source_folder, destination_folder):
    """
    Copia una carpeta clonada a un destino local.

    :param source_folder: Ruta de la carpeta clonada.
    :param destination_folder: Ruta de destino donde se copiará la carpeta.
    """
    if not os.path.exists(source_folder):
        print(f"La carpeta fuente no existe: {source_folder}")
        return

    if os.path.exists(destination_folder):
        print(f"La carpeta destino ya existe: {destination_folder}")
        return

    try:
        # Copiar la carpeta completa
        shutil.copytree(source_folder, destination_folder)
        print(f"Carpeta copiada exitosamente de '{source_folder}' a '{destination_folder}'")
    except Exception as e:
        print(f"Error al copiar la carpeta: {e}")

In [ ]:
#   Mover imágenes

def move_images_to_parent_and_delete_subfolders(parent_folder):
    """
    Mueve todas las imágenes de las subcarpetas a la carpeta madre y elimina las subcarpetas.

    :param parent_folder: Ruta de la carpeta madre que contiene las subcarpetas.
    """
    for root, subdirs, files in os.walk(parent_folder, topdown=False):  # Recorre de abajo hacia arriba
        for file in files:
            # Verificar si es una imagen válida
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                source_path = os.path.join(root, file)
                destination_path = os.path.join(parent_folder, file)
                
                # Mover el archivo, renombrar si hay conflicto
                if os.path.exists(destination_path):
                    base, ext = os.path.splitext(file)
                    new_name = f"{base}_copy{ext}"
                    destination_path = os.path.join(parent_folder, new_name)
                
                shutil.move(source_path, destination_path)
                print(f"Movido: {source_path} -> {destination_path}")

        # Eliminar la subcarpeta si está vacía
        if root != parent_folder:  # No eliminar la carpeta madre
            try:
                os.rmdir(root)
                print(f"Eliminada carpeta: {root}")
            except OSError:
                print(f"No se pudo eliminar {root} (posiblemente no está vacía)")

In [ ]:
#   Convertir las imágenes .png en formato .jpg

def convert_png_to_jpg(input_dir, output_dir):
    """
    Convierte todas las imágenes PNG en una carpeta (y sus subcarpetas) a formato JPG.

    :param input_dir: Carpeta raíz con imágenes PNG.
    :param output_dir: Carpeta donde se guardarán las imágenes JPG convertidas.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, _, files in os.walk(input_dir):
        # Crear la misma estructura de carpetas en el directorio de salida
        relative_path = os.path.relpath(root, input_dir)
        save_path = os.path.join(output_dir, relative_path)
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        for file in files:
            if file.lower().endswith(".png"):  # Solo procesar archivos PNG
                file_path = os.path.join(root, file)
                try:
                    # Abrir imagen PNG
                    with Image.open(file_path) as img:
                        # Convertir a modo RGB (necesario para JPG)
                        rgb_img = img.convert("RGB")
                        # Guardar como JPG
                        jpg_path = os.path.join(save_path, os.path.splitext(file)[0] + ".jpg")
                        rgb_img.save(jpg_path, "JPEG")
                        print(f"Convertido: {file_path} -> {jpg_path}")
                except Exception as e:
                    print(f"Error al procesar {file_path}: {e}")

In [ ]:
#   Borrar las carpetas inútiles

def delete_folder(folder_path):
    """
    Elimina una carpeta y todas sus subcarpetas y archivos.

    :param folder_path: Ruta de la carpeta a eliminar.
    """
    if os.path.exists(folder_path):
        try:
            shutil.rmtree(folder_path)
            print(f"Carpeta eliminada exitosamente: {folder_path}")
        except Exception as e:
            print(f"Error al eliminar la carpeta: {e}")
    else:
        print(f"La carpeta no existe: {folder_path}")

In [ ]:
#   Creación de un dataframe con las imágenes

def process_images_to_dataframe(image_folder):
    """
    Crea un DataFrame a partir de las imágenes en una carpeta.
    Cada fila contiene los valores de píxeles de una imagen, la clase derivada del nombre del archivo,
    y usa la parte inicial del nombre del archivo como índice.

    :param image_folder: Ruta a la carpeta con imágenes.
    :return: DataFrame con valores de píxeles, clase e índice personalizado.
    """
    data = []

    for file in os.listdir(image_folder):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Ruta completa del archivo
            file_path = os.path.join(image_folder, file)
            
            # Cargar la imagen y convertirla a escala de grises
            with Image.open(file_path) as img:
                img = img.convert('L')  # Convertir a escala de grises
                img_array = np.array(img).flatten()  # Aplanar la imagen
            
            # Extraer clase y nombre del índice
            try:
                name_part = file.split("_")[0]
                class_part = file.split("_")[1].split(".")[0]
            except IndexError:
                print(f"Formato de nombre inesperado: {file}")
                continue

            # Crear fila con valores de píxeles y clase
            data.append([name_part, class_part, *img_array])

    # Crear el DataFrame
    column_names = ['indice', 'clase'] + [f'pixel_{i}' for i in range(len(img_array))]
    df = pd.DataFrame(data, columns=column_names)

    # Establecer la columna 'indice' como índice del DataFrame
    df.set_index('indice', inplace=True)

    return df

In [ ]:
#   Ampliar el número de imágenes

def augment_and_expand_dataset(df, image_folder, min_instances=20):
    """
    Aumenta el número de imágenes para las clases con menos de 'min_instances' instancias,
    y las integra al DataFrame original.
    
    :param df: DataFrame original con columnas ['clase', 'pixel_0', ..., 'pixel_n'].
    :param image_folder: Ruta a la carpeta donde se almacenan las imágenes originales.
    :param min_instances: Número mínimo deseado de instancias por clase.
    :return: DataFrame actualizado con las nuevas imágenes.
    """
    new_data = []
    class_counts = df['clase'].value_counts()

    for clase, count in class_counts.items():
        if count < min_instances:
            # Número de imágenes adicionales necesarias
            num_to_add = min_instances - count

            # Filtrar imágenes de la clase actual
            class_images = df[df['clase'] == clase].index.tolist()
            
            for _ in range(num_to_add):
                # Seleccionar una imagen aleatoria de la clase
                random_index = random.choice(class_images)
                random_image_name = random_index + f"_{clase}.jpg"
                random_image_path = os.path.join(image_folder, random_image_name)
                
                # Cargar la imagen original
                with Image.open(random_image_path) as img:
                    # Aplicar transformación sencilla para simular una imagen nueva (ejemplo: voltear horizontalmente)
                    augmented_image = ImageOps.mirror(img)

                    # Crear un nuevo nombre para la imagen
                    random_digits = ''.join(random.choices("0123456789", k=6))
                    new_image_name = f"{random_digits}_{clase}.jpg"
                    new_image_path = os.path.join(image_folder, new_image_name)

                    # Guardar la nueva imagen
                    augmented_image.save(new_image_path)
                
                # Convertir la imagen aumentada a un vector de píxeles
                img_array = np.array(augmented_image.convert('L')).flatten()

                # Crear nueva fila para el DataFrame
                new_row = [random_digits, clase, *img_array]
                new_data.append(new_row)

    # Crear DataFrame con los nuevos datos
    if new_data:
        column_names = ['indice', 'clase'] + [f'pixel_{i}' for i in range(len(new_data[0]) - 2)]
        new_df = pd.DataFrame(new_data, columns=column_names)
        new_df.set_index('indice', inplace=True)
        df = pd.concat([df, new_df])

    return df